# Trading env tester
### This notebook explores an openai trading env for BTC ohcv data
<hr>

#### -Pulls custom signal and historical data from BWA portmaker module
##### -Applies ML and RL to trading env for backtesting


### Imports
<hr>

In [1]:
import random
import json
import gym
from gym import spaces
import pandas as pd
import numpy as np
import datetime as dt
#from env.TradingEnv import TradingEnv
import portmaker
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [80]:
import plotly.graph_objects as go

from plotly.subplots import make_subplots
import plotly.tools as tls
import plotly

def plot_signal(df,save_plot=False):
    
    xs = df.index
    #print(signal.iloc[0])
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=xs, y=df['Close'], name="Close"))

    fig.add_trace(go.Scatter(x=xs, y=df['Position'], name="Signal",)
    ,secondary_y=True)
    # Set x-axis title
    fig.update_xaxes(title_text="Date")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Price</b> ", secondary_y=False)
    fig.update_yaxes(title_text="<b>Signal</b>", secondary_y=True)
    return fig
def plot_balances(df,save_plot=False):
    df = df.copy()
    df = df.sort_values('Date')
    df.set_index("Date",inplace=True)
    df = df.iloc[3:]
    xs = df.index                                           
    #create traces
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=xs, y=df['USDT'], name="USDT Balance"),secondary_y=True)
    fig.add_trace(go.Scatter(x=xs, y=df['BTC'], name="BTC Balance", line_color='red'))
    
    fig.update_layout(title_text='Balances',
                  xaxis_rangeslider_visible=True)

    fig.update_xaxes(title_text="Date")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>BTC Balance</b> ", secondary_y=False)
    fig.update_yaxes(title_text="<b>USDT Balance</b> ", secondary_y=True)
    return df,fig

def plot_returns(df,save_plot=False):
    df = df.copy()
    df = df.sort_values('Date')
    df.set_index("Date",inplace=True)
    df = df.iloc[3:]
    xs = df.index
    #infer postion after env overtrading lim
    df["Inv Position"] = df["USDT"].diff() 
    df["Inv Position"] = np.where(df['Inv Position']>0,1.0,-1.0)
    #create returns
    df['BTC Market Return'] = np.log(df["Price"] / df["Price"].shift(1))
    df['Strategy USDT Return'] = np.log(df["USDT"] / df["USDT"].shift(1))
    df['Strategy BTC Return'] = np.log(df["BTC"] / df["BTC"].shift(1))
    #all_sigs['Strategy2 Return'] = all_sigs['Market Return'] * all_sigs['Position2']
    #all_sigs['Strategy3 Return'] = all_sigs['Market Return'] * all_sigs['Position3']
                                                                       
    #create traces
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=xs, y=df['Price'], name="Close prices"),secondary_y=True)
    fig.add_trace(go.Scatter(x=xs, y=df['BTC Market Return'].cumsum()*100, name="BTC Market Returns", line_color='red'))
    fig.add_trace(go.Scatter(x=xs, y=df['Strategy BTC Return'].cumsum()*100, name="Strategy BTC Returns", line_color='deepskyblue'))
    fig.add_trace(go.Scatter(x=xs, y=df['Strategy USDT Return'].cumsum()*100, name="Strategy USDT Returns", line_color='green'))

    fig.update_layout(title_text='Returns',
                  xaxis_rangeslider_visible=True)
    fig.update_xaxes(title_text="Date")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Log Returns</b> ", secondary_y=False)
    fig.update_yaxes(title_text="<b>BTC Price</b> ", secondary_y=True)

    return df,fig



## Test trading gym env class 
<hr>

In [69]:

MAX_STEPS = 20000
class TradingEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, df):
        super(TradingEnv, self).__init__()
        self.last_sig = 0
        self.usdt_holdings = 1000
        self.btc_holdings = 0
        self.current_step = 0
        self.current_price = 0
        self.prev_usdt,self.prev_btc = 0,0
        #self.init_btc = self.btc_holdings
        self.max_btc = 0
        self.max_usdt = 0
        self.btc_returns = 0
        self.current_act = 0
        self.trades = pd.DataFrame()
        self.balances = pd.DataFrame()
        self.dates = df["Close time"]
        self.last_entry = 0
        self.last_exit = 0
        self.last_entry_t = 0
        self.last_exit_t = 0
        self.reward = 0
        

        self.df = df
        # Actions of the format Buy x%, Sell x%, Hold, etc.
        self.action_space = spaces.Box(
            low=np.array([0, 0]), high=np.array([3, 1]), dtype=np.float16)

        # Prices contains the OHCL values for the last five prices
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(6, 6), dtype=np.float16)
        

    def _next_observation(self):
        # Get the hist data at step
        frame = np.array([
            self.df["Open"].iloc[self.current_step],
            self.df["High"].iloc[self.current_step],
            self.df["Low"].iloc[self.current_step],
            self.df["Close"].iloc[self.current_step],
            self.df["Volume"].iloc[self.current_step],
            #self.df["Position"].iloc[self.current_step],
        ])
        return frame

    def _take_action(self, action):
        #nested buy sell functions
        def buy():
            total_possible = float(self.usdt_holdings / self.current_price)
            #update trade df
            trade_stamp = portmaker.to_dt(self.df["Close time"].iloc[self.current_step])
            trade = pd.DataFrame({"Date":trade_stamp,"USDT":int(self.usdt_holdings),"BTC":total_possible,"Type":"BUY","Price":self.current_price},index=[0])
            self.trades = pd.concat([self.trades,trade])
            
            #set last entry to current price
            self.last_entry = self.current_price
            # Buy amount % of balance in shares
            
            #deduct spent usdt from usdt_holdings 
            self.usdt_holdings = self.usdt_holdings -(total_possible*self.current_price)
            #update balances
            
            
            total_possible = round(total_possible,5)
            #update balances
            self.btc_holdings = total_possible
            bal = pd.DataFrame({"Date":trade_stamp,"USDT":int(self.usdt_holdings),"BTC":self.btc_holdings,"Price":self.current_price},index=[0])
            self.balances = pd.concat([self.balances,bal])
            #update max btc held
            if total_possible > self.max_btc:
                self.max_btc = self.btc_holdings
            print("Buy {}btc at {}\t{}".format(total_possible,str(self.current_price),trade_stamp))
            
        def sell():
            trade_stamp = portmaker.to_dt(self.df["Close time"].iloc[self.current_step])
            self.last_exit = self.current_price
            total_possible = float(self.btc_holdings)#/float(last_price["bidPrice"])
            print("Sell {} btc at {}\t{}".format(total_possible,df["Close"].iloc[self.current_step],trade_stamp))
            #update balances
            self.btc_holdings = self.btc_holdings -self.btc_holdings
            self.usdt_holdings = float(total_possible*self.current_price)
            #update trade df
            #trade_stamp = portmaker.to_dt(self.df["Close time"].iloc[self.current_step])
            trade = pd.DataFrame({"Date":trade_stamp,"USDT":int(self.usdt_holdings),"BTC":total_possible,"Type":"SELL","Price":self.current_price},index=[0])
            bal = pd.DataFrame({"Date":trade_stamp,"USDT":int(self.usdt_holdings),"BTC":self.btc_holdings,"Price":self.current_price},index=[0])
            self.balances = pd.concat([self.balances,bal])    
            self.trades = pd.concat([self.trades,trade])    
            
        # Set the current price to a random price within the current time step
        self.current_price = random.uniform(self.df["Open"].iloc[self.current_step], self.df["Close"].iloc[self.current_step])
        #print(current_price,self.df["Open"].iloc[self.current_step])
        action_type = action[0]
        self.current_act = action_type
        #external purchase quantity
        amount = action[1]
        """
        BUY BTC
        ==============
        Conditions are:
        -if action is buy
        -has usdt to buy btc
        -current signal is not a same as previous signal
        -has no btc
        ==============
        """
        
            
            
        if (action_type > 0 and self.usdt_holdings !=0 and self.last_sig!=action_type and self.btc_holdings ==0):
            #overtrading limiter 2
            td = self.current_price/self.last_exit
            if(td > .99 and td < 1.01):
                pass
                #print("no trades", self.df.index.values[self.current_step],self.current_step)
            else:
                buy()
            

        elif (action_type < 0 and self.btc_holdings !=0 and self.last_sig!=action_type):
            # Buy amount % of balance in shares
            td = self.current_price/self.last_entry
            #prevent action at less the +-1% delta
            #backtest a multitude of these tolerances to tune R/R
            if(td > .99 and td < 1.01):
                pass
                #uncomment to see possible overtrade
                #print("no trades", self.df.index.values[self.current_step],self.current_step)
                #df["Position"].iloc[self.current_step]
            else:
                sell()
                
                
                
        
            if self.usdt_holdings > self.max_usdt:
                self.max_usdt = self.usdt_holdings
            #print("Selling {}btc at {}".format(total_possible,str(current_price)))
            #print("{} remaining usdt".format(self.usdt_holdings))
            #print("\n====\n")
            
            
        self.last_sig  = action[0]
        if(self.btc_holdings!=0):
            self.btc_net = (self.btc_holdings/self.init_btc)*100

        if self.btc_holdings > self.max_btc:
            self.max_btc = self.btc_holdings
        if self.btc_holdings>0:
            curr_profit = (self.btc_holdings/ self.init_btc)
            #print(self.btc_holdings,self.init_btc)
        else:
            #print("USDT: {}  Init btc: {}".format(self.usdt_holdings,self.init_btc))
            
                #print("USDT: {}  Init btc: {}".format(self.usdt_holdings,self.init_btc))
            self.prev_usdt,self.prev_btc = self.usdt_holdings,self.btc_holdings
            #print("Current profit: ",curr_profit)"""


    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        if self.current_step > len(self.df['Open'].values) - 6:
            self.current_step = 0

        delay_modifier = (self.current_step / MAX_STEPS)
        if(self.btc_holdings!=0):
            self.reward = self.btc_holdings/self.init_btc
        elif(self.btc_holdings==0 and self.usdt_holdings!=0):
            curr_btc = float(self.usdt_holdings / self.current_price)
            self.reward = curr_btc/self.init_btc
            
        done = (self.btc_holdings/self.init_btc == 0 and self.usdt_holdings==0)
        if(self.current_step == len(df)-1):
            done = True

        obs = self._next_observation()

        return obs, self.reward, done, {}

    def reset(self):
        # Reset the state of the environment to an initial state
        self.usdt_holdings = 1000
        self.btc_holdings = 0
        self.max_usdt = self.usdt_holdings
        self.max_btc = 0
        self.max_usdt = 0
        self.btc_returns = 0
        self.current_act = 0
        self.current_price = 0
        self.init_btc = (self.usdt_holdings/df['Open'].iloc[self.current_step])
        self.init_usdt = self.usdt_holdings
        self.trades = pd.DataFrame()
        self.balances = pd.DataFrame()
        self.last_entry = 0
        self.last_exit = 0
        self.last_entry_t = 0
        self.last_exit_t = 0
        self.reward = 0

        # Set the current step to a random point within the data frame
        #print(self.df.columns)
        self.current_step = 0#random.randint(0, len(self.df['Open'].values) - 6)

        return self._next_observation()

    def render(self, mode='human', close=False):
        curr_profit = self.reward
        # Render the environment to the screen
        #profit = (self.btc_holdings/ self.init_btc)
        if(self.reward>0):
            curr_profit*=100
        else:
            profit = (1-curr_profit)*100
        max_curr = (self.max_btc/self.init_btc)*100
        
        
        print(f'Step: {self.current_step}')
        print("Timestamp: ",portmaker.to_dt(self.df["Close time"].iloc[self.current_step]))
        print("-------------")
        print(f'Current BTC Balance: {self.btc_holdings}')
        print("Init BTC: ",self.init_btc)
        print("-------------")
        print("Current USDT Balance: {}".format(self.usdt_holdings))
        print("Init USDT: ",self.init_usdt)
        print("-------------")
        print("-------------")
        print('Max usdt: {}'.format(self.max_usdt))
        print(f'Max btc: {self.max_btc}')
        print('Max profit at step: {}'.format(max_curr))
        #print(f'Profit: {profit}')
        if(self.current_step == len(df)-8):
            make_plots()
        #print(df.iloc[self.current_step].index)

## Get data and signal from portmaker module
<hr>

#####  -Applies BWA signal to ohlcv df for optional ML optimization
##### -Gets data from -n*day to current time using unixtime time for serve communication</b>



# The algorithms require a vectorized environment to run


# Creating the Env with OHLCV data frame
<hr>

#### -Model fitting and training called here
##### -Acces live balances via env.trades

In [85]:
"""
end = int(portmaker.current_milli_time())
d_intv = 300
st = end-(86400000*d_intv)
coin = "BTCUSDT"
candle_k='5m'
data = portmaker.get_data(coin,st,end,candle_k)
t1 = portmaker.current_milli_time()
all_prices,df = portmaker.get_intv(data,coin,d_intv,candle_k)
#print(df.columns)
all_sigs = df.set_index("Close time")
print(all_sigs.head(3))
"""
#get live bot data
!scp -r root@159.203.174.22:/root/BWA/out/btc_signal.csv /home/me/Documents/BWA_dash/
live_sig = pd.read_csv("/home/me/Documents/BWA_dash/btc_signal.csv")
live_sig = live_sig.drop_duplicates()
#print(live_sig.tail(3))
df = live_sig.set_index("Open time")
#"""

btc_signal.csv                                100% 1400KB   1.7MB/s   00:00    


In [92]:
df = pd.read_csv('/home/me/Documents/BWA_dash/btc_hist_sig.csv')
#df = pd.read_csv("/home/me/Documents/BWA_dash/local_sig.csv")
env = TradingEnv(df[:50000])
print("Action shape: ",env.action_space.shape)
print("State shape: ",env.observation_space.shape)

#specify max length of each episode
max_iter = len(df)+6
#reset env to default state
#starts with $1000 ...will edit for input
obs = env.reset()
rc = 1
trades = None
dates = env.dates
lt = None
balances = None
#loop through iterations
for i in range(max_iter):
    #random agent
    rand_agent = np.random.randint(-1,2)
    #BWA signal agent/actor
    BWA_sig = env.df.Position.iloc[env.current_step]
    trades = env.trades
    balances = env.balances
    
    if(len(trades)%2==0 and len(trades)!= lt):
    #if(len(trades)!= lt):
        lt = len(trades)
        #print(trades.columns)
        print("\n========================\n")
        print("{} trades taken".format(int(lt/2)))
    if (i%1000==0):
        balances =  env.render()
    
    #random agent for testing
    #obs, done, info = env.step([rand_agent,1])
    
    #portmaker signal for benchmark
    #obs,reward, done, info = env.step([BWA_sig,1])
    
    #Q Learning optimized portmaker
    obs,reward, done, info = env.step([BWA_sig,1])
    rc+=1
    

Action shape:  (2,)
State shape:  (6, 6)


0 trades taken
Step: 0
Timestamp:  2018-10-26 09:29:59.999000
-------------
Current BTC Balance: 0
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 1000
Init USDT:  1000
-------------
-------------
Max usdt: 0
Max btc: 0
Max profit at step: 0.0
Buy 0.1526btc at 6553.245506546758	2018-10-26 10:14:59.999000
Sell 0.1526 btc at 6485.28	2018-10-29 04:04:59.999000


1 trades taken
Buy 0.15567btc at 6358.0383476032	2018-10-29 10:19:59.999000
Step: 1000
Timestamp:  2018-10-29 20:44:59.999000
-------------
Current BTC Balance: 0.15567
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 0.0
Init USDT:  1000
-------------
-------------
Max usdt: 989.7855273431128
Max btc: 0.15567
Max profit at step: 101.94439125
Sell 0.15567 btc at 6424.11	2018-11-01 19:44:59.999000


2 trades taken
Buy 0.15556btc at 6431.279114387743	2018-11-01 20:04:59.999000
Step: 2000
Timestamp:  2018-11-02 07:59:59.999000
-------------
Current BTC Ba



32 trades taken
Buy 0.19113btc at 4278.327864398212	2018-12-01 18:54:59.999000
Sell 0.19113 btc at 4200.02	2018-12-01 19:59:59.999000


33 trades taken
Buy 0.1911btc at 4204.554703475204	2018-12-01 20:04:59.999000
Sell 0.1911 btc at 4155.61	2018-12-01 23:04:59.999000


34 trades taken
Buy 0.19125btc at 4154.427138970296	2018-12-01 23:14:59.999000
Sell 0.19125 btc at 4230.01	2018-12-02 12:49:59.999000


35 trades taken
Buy 0.1906btc at 4231.179369239514	2018-12-02 13:09:59.999000
Sell 0.1906 btc at 4139.99	2018-12-02 16:09:59.999000


36 trades taken
Buy 0.19107btc at 4145.540191126261	2018-12-02 16:14:59.999000
Step: 11000
Timestamp:  2018-12-03 19:09:59.999000
-------------
Current BTC Balance: 0.19107
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 0.0
Init USDT:  1000
-------------
-------------
Max usdt: 1021.7496209930796
Max btc: 0.19149
Max profit at step: 125.40201375
Sell 0.19107 btc at 3930.53	2018-12-03 22:19:59.999000


37 trades taken
Buy 0.19106btc at

Sell 0.21828 btc at 3814.59	2019-01-02 21:24:59.999000


68 trades taken
Buy 0.21861btc at 3816.4441546544936	2019-01-02 21:29:59.999000
Sell 0.21861 btc at 3766.78	2019-01-03 15:59:59.999000


69 trades taken
Buy 0.21965btc at 3747.135393925841	2019-01-03 16:44:59.999000
Step: 20000
Timestamp:  2019-01-04 00:14:59.999000
-------------
Current BTC Balance: 0.21965
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 0.0
Init USDT:  1000
-------------
-------------
Max usdt: 1021.7496209930796
Max btc: 0.21965
Max profit at step: 143.84329375
Sell 0.21965 btc at 3797.46	2019-01-04 22:39:59.999000


70 trades taken
Buy 0.21955btc at 3807.530420216174	2019-01-05 03:44:59.999000
Sell 0.21955 btc at 3991.84	2019-01-06 15:54:59.999000


71 trades taken
Buy 0.21922btc at 3998.716292802643	2019-01-06 16:04:59.999000
Step: 21000
Timestamp:  2019-01-07 11:29:59.999000
-------------
Current BTC Balance: 0.21922
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 0.0
I

Sell 0.22817 btc at 4112.5	2019-02-23 15:49:59.999000


93 trades taken
Buy 0.22808btc at 4116.139382799806	2019-02-23 15:54:59.999000
Sell 0.22808 btc at 4159.03	2019-02-23 20:39:59.999000


94 trades taken
Buy 0.22753btc at 4173.321717271754	2019-02-23 22:09:59.999000
Step: 35000
Timestamp:  2019-02-25 00:49:59.999000
-------------
Current BTC Balance: 0.22753
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 0.0
Init USDT:  1000
-------------
-------------
Max usdt: 1021.7496209930796
Max btc: 0.23584
Max profit at step: 154.44572
Sell 0.22753 btc at 3811.05	2019-02-26 15:49:59.999000


95 trades taken
Buy 0.22815btc at 3799.960024680643	2019-02-26 16:49:59.999000
Sell 0.22815 btc at 3724.68	2019-02-27 12:34:59.999000


96 trades taken
Buy 0.22124btc at 3860.276167396186	2019-02-28 07:44:59.999000
Step: 36000
Timestamp:  2019-02-28 12:04:59.999000
-------------
Current BTC Balance: 0.22124
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 0.0
Init U

Step: 48000
Timestamp:  2019-04-11 09:54:59.999000
-------------
Current BTC Balance: 0.22445
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 0.0
Init USDT:  1000
-------------
-------------
Max usdt: 1191.0858707172858
Max btc: 0.23584
Max profit at step: 154.44572
Sell 0.22445 btc at 5075.08	2019-04-12 22:44:59.999000


121 trades taken
Buy 0.22433btc at 5078.206684207293	2019-04-12 22:49:59.999000
Step: 49000
Timestamp:  2019-04-14 21:09:59.999000
-------------
Current BTC Balance: 0.22433
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 0.0
Init USDT:  1000
-------------
-------------
Max usdt: 1191.0858707172858
Max btc: 0.23584
Max profit at step: 154.44572
Sell 0.22433 btc at 5182.0	2019-04-16 23:39:59.999000


122 trades taken
Buy 0.22421btc at 5187.676170890891	2019-04-17 00:49:59.999000
Sell 0.22421 btc at 6549.51	2018-10-26 09:49:59.999000


123 trades taken
Step: 5
Timestamp:  2018-10-26 09:54:59.999000
-------------
Current BTC Balance:

Sell 0.26734 btc at 4186.03	2018-11-29 13:09:59.999000


152 trades taken
Buy 0.26531btc at 4218.262489708609	2018-11-29 13:34:59.999000
Sell 0.26531 btc at 4333.1	2018-11-29 21:09:59.999000


153 trades taken
Step: 10005
Timestamp:  2018-11-30 08:19:59.999000
-------------
Current BTC Balance: 0.0
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 1148.7439893112378
Init USDT:  1000
-------------
-------------
Max usdt: 1500.176854727551
Max btc: 0.27165
Max profit at step: 177.89679375
Buy 0.28415btc at 4042.6980930863647	2018-12-01 00:34:59.999000
Sell 0.28415 btc at 4294.13	2018-12-01 08:24:59.999000


154 trades taken
Buy 0.29049btc at 4191.0862925198035	2018-12-01 16:04:59.999000
Sell 0.29049 btc at 4271.67	2018-12-01 18:44:59.999000


155 trades taken
Buy 0.28969btc at 4278.350860554398	2018-12-01 18:54:59.999000
Sell 0.28969 btc at 4200.02	2018-12-01 19:59:59.999000


156 trades taken
Buy 0.28939btc at 4205.147321196257	2018-12-01 20:04:59.999000
Sell 0.28939 bt

Sell 0.32195 btc at 3764.0	2018-12-29 08:04:59.999000


189 trades taken
Buy 0.32025btc at 3799.876915121741	2018-12-29 12:09:59.999000
Sell 0.32025 btc at 3844.93	2018-12-29 14:54:59.999000


190 trades taken
Buy 0.32051btc at 3844.9420252272726	2018-12-29 14:59:59.999000
Sell 0.32051 btc at 3744.11	2018-12-30 18:39:59.999000


191 trades taken
Buy 0.3204btc at 3745.954123250019	2018-12-30 19:39:59.999000
Step: 19005
Timestamp:  2018-12-31 13:29:59.999000
-------------
Current BTC Balance: 0.3204
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 0.0
Init USDT:  1000
-------------
-------------
Max usdt: 1500.176854727551
Max btc: 0.32846
Max profit at step: 215.10024249999998
Sell 0.3204 btc at 3702.9	2018-12-31 15:59:59.999000


192 trades taken
Buy 0.32049btc at 3701.4136809608563	2018-12-31 16:44:59.999000
Sell 0.32049 btc at 3814.59	2019-01-02 21:24:59.999000


193 trades taken
Buy 0.32077btc at 3819.443739778576	2019-01-02 21:29:59.999000
Sell 0.32077 btc at 3766

Sell 0.35538 btc at 3875.08	2019-02-18 17:44:59.999000


216 trades taken
Buy 0.35369btc at 3897.987170321197	2019-02-18 20:09:59.999000
Sell 0.35369 btc at 3932.42	2019-02-19 13:44:59.999000


217 trades taken
Buy 0.35259btc at 3953.604716109223	2019-02-20 02:24:59.999000
Step: 34005
Timestamp:  2019-02-21 13:59:59.999000
-------------
Current BTC Balance: 0.35259
Init BTC:  0.15270089711777057
-------------
Current USDT Balance: 0.0
Init USDT:  1000
-------------
-------------
Max usdt: 1500.176854727551
Max btc: 0.35704
Max profit at step: 233.81657
Sell 0.35259 btc at 4112.5	2019-02-23 15:49:59.999000


218 trades taken
Buy 0.35259btc at 4114.224161621382	2019-02-23 15:54:59.999000
Sell 0.35259 btc at 4159.03	2019-02-23 20:39:59.999000


219 trades taken
Buy 0.3521btc at 4173.9164783999	2019-02-23 22:09:59.999000
Sell 0.3521 btc at 4122.7	2019-02-24 04:19:59.999000


220 trades taken
Buy 0.35188btc at 4127.778746471204	2019-02-24 04:24:59.999000
Step: 35005
Timestamp:  2019-02-25 0

# Visualize Agent outputs
<hr>
<br>
<b>-Trades, and Acc balances</b>

In [90]:



#fig = plot_signal(df)
#fig.show()
agent_trades,fig = plot_returns(trades[1:])
fig.show()
agent_balances,fig = plot_balances(balances[1:])
fig.show()
print(agent_trades.head(10))
#print(trades.values[:,0])
#for i in trades.values:
#    print(i[0])
#trades["USDT"]

                         USDT       BTC  Type        Price  Inv Position  \
Date                                                                       
2019-04-18 08:49:59.999  1349  0.257105   BUY  5247.626717          -1.0   
2019-04-20 07:09:59.999  1010  0.190970  SELL  5289.241238          -1.0   
2019-04-20 08:09:59.999  1010  0.190982   BUY  5288.914901          -1.0   
2019-04-21 00:24:59.999  1777  0.335480  SELL  5297.460415           1.0   
2019-04-21 00:39:59.999  1777  0.335237   BUY  5301.297453          -1.0   
2019-04-21 00:44:59.999  1362  0.257100  SELL  5301.001795          -1.0   
2019-04-21 02:39:59.999  1758  0.335240  SELL  5246.965070           1.0   
2019-04-21 08:44:59.999  1758  0.336052   BUY  5234.280915          -1.0   
2019-04-21 09:04:59.999   999  0.190980  SELL  5234.563234          -1.0   
2019-04-21 09:44:59.999   999  0.190955   BUY  5235.247325          -1.0   

                         BTC Market Return  Strategy USDT Return  \
Date               

In [91]:
agent_trades

,USDT,BTC,Type,Price,Inv Position,BTC Market Return,Strategy USDT Return,Strategy BTC Return
Date,,,,,,,,
2019-04-18 08:49:59.999,1349,0.257105,BUY,5247.626717,-1.0,NaN,NaN,NaN
2019-04-20 07:09:59.999,1010,0.190970,SELL,5289.241238,-1.0,0.007899,-0.289413,-0.297368
2019-04-20 08:09:59.999,1010,0.190982,BUY,5288.914901,-1.0,-0.000062,0.000000,0.000062
2019-04-21 00:24:59.999,1777,0.335480,SELL,5297.460415,1.0,0.001614,0.564976,0.563384
2019-04-21 00:39:59.999,1777,0.335237,BUY,5301.297453,-1.0,0.000724,0.000000,-0.000724
2019-04-21 00:44:59.999,1362,0.257100,SELL,5301.001795,-1.0,-0.000056,-0.265972,-0.265373
2019-04-21 02:39:59.999,1758,0.335240,SELL,5246.965070,1.0,-0.010246,0.255223,0.265382
2019-04-21 08:44:59.999,1758,0.336052,BUY,5234.280915,-1.0,-0.002420,0.000000,0.002420
2019-04-21 09:04:59.999,999,0.190980,SELL,5234.563234,-1.0,0.000054,-0.565177,-0.565098


In [13]:
df.to_csv('/home/me/Documents/BWA_dash/btc_hist_sig.csv')